In [1]:
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install nbformat==4.2.0
!mamba install html5lib==1.1 -y 
!pip install lxml==4.6.4

  Using cached yfinance-0.1.67-py2.py3-none-any.whl (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 25.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 37.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 20.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 25.3 MB/s eta 0:00:00
zsh:1: command not found: mamba
  Using cached nbformat-4.2.0-py2.py3-none-any.whl (153 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 676.6 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 995.1 kB/s eta 0:00:000:00:01
   ━━━━━

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [5]:
tesla=yf.Ticker("TSLA")

In [6]:
tesla_data=tesla.history(period="max")

AttributeError: 'Index' object has no attribute 'tz_localize'

In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

In [ ]:
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data=requests.get(url).text

In [ ]:
soup=BeautifulSoup(html_data,'html5lib')

In [ ]:
tables = soup.find_all('table')

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[1].find("tbody").find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
    
    tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

In [ ]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.tail()

In [ ]:
gme=yf.Ticker("GME")

In [ ]:
gme_data=gme.history(period="max")

In [ ]:
gme_data.reset_index(inplace=True)
gme_data.head()

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(url).text

In [ ]:
soup = BeautifulSoup(html_data, "html.parser")

In [ ]:
soup.find_all("tbody")[1]

gme_revenue = pd.DataFrame(columns=['Date','Revenue'])
for row in soup.find_all("tbody")[1].find_all('tr'): #find all table rows using the tr tag
    col = row.find_all("td") #set the column variable equal to the cell content for each row. find
    date = col[0].text #for each row (tr tag) the date is equal to the first td (index zero of col)
    revenue = col[1].text #for each row, the open value is qual to the second td (index 1 of col)
    
    # Finally we concat the data of each row to the table
    #pandas depricated the ability to append to a dataframe, so we create one dataframe, passing in a python dictionary
            #we want the Date column to pull the date variable, etc.
            #index = [1] tells python to start our table at 
    
    gme_rev = pd.DataFrame({"Date":date, "Revenue":revenue},index =[0])    
    #then we concat the gme_rev table to our empty dataframe we created earlier, gme_revenue
    gme_revenue = pd.concat([gme_revenue,gme_rev])

In [ ]:
gme_revenue.reset_index(drop=True, inplace=True)

In [ ]:
gme_revenue.head()

In [ ]:
gme_revenue.tail()

In [ ]:
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")

In [ ]:
gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [ ]:
gme_revenue.tail()

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop')